# FUA simplification with [`cityseer`](https://github.com/benchmark-urbanism/cityseer-api)

* **Simons, G.** (2023). *The cityseer Python package for pedestrian-scale network-based urban analysis*. Environment and Planning B: Urban Analytics and City Science, 50(5), 1328-1344. https://doi.org/10.1177/23998083221133827

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-11-03T19:40:24.059720-05:00

Python implementation: CPython
Python version       : 3.11.10
IPython version      : 8.28.0

Compiler    : Clang 17.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import pathlib
import time

import cityseer
import geopandas

from core import utils

%watermark -w
%watermark -iv

Watermark: 2.5.0

core     : 0.1.dev124+g2144fe2
geopandas: 1.0.1
cityseer : 4.15.2



In [3]:
def run_cityseer(city: str) -> geopandas.GeoDataFrame:
    """Run the ``cityseer`` workflow as laid out in
    [https://benchmark-urbanism.github.io/cityseer-examples/examples/graph_cleaning.html#manual-cleaning]
    with ***all default arguments***.

    Parameters
    ----------
    city : str
        City anme. See ``utils.city_fua.keys()``.

    Returns
    -------
    geopandas.GeoDataFrame
        Resultant simplified network edges – only geometry column.
        Will be written to ``.parquet``.
    """
    edges = utils.read_original(city)
    graph = cityseer.tools.io.nx_from_generic_geopandas(edges)
    graph = cityseer.tools.graphs.nx_remove_dangling_nodes(graph)
    graph = cityseer.tools.graphs.nx_split_opposing_geoms(graph)
    graph = cityseer.tools.graphs.nx_consolidate_nodes(graph)
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)
    graph = cityseer.tools.graphs.nx_iron_edges(graph)
    return cityseer.tools.io.geopandas_from_nx(graph, crs=edges.crs).rename_geometry(
        "geometry"
    )[["geometry"]]

In [4]:
output_base = pathlib.Path("..", "..", "data")

In [5]:
global_start = time.time()

for city, fua in utils.city_fua.items():
    print("===================================================================")
    print(f"Starting {city}")
    print("-------------------------")
    print("\n\n")

    fua_start = time.time()
    simplified = run_cityseer(city)
    output_fua = output_base / pathlib.Path(f"{fua}", "cityseer", f"{fua}.parquet")
    simplified.to_parquet(output_fua)
    fua_end = time.time()
    fua_duration = round((fua_end - fua_start) / 60.0, 3)

    print(f"\t{city} completed in: {fua_duration} minutes")
    print("\n\n\n\n")

print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")
print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")

global_end = time.time()
global_duration = round((global_end - global_start) / 60.0, 3)

print(f"Total runtime: {global_duration} minutes")

Starting Aleppo
-------------------------





100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78908/78908 [00:03<00:00, 23699.61it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78908/78908 [00:00<00:00, 357442.79it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64285/64285 [00:00<00:00, 1392598.96it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78572/78572 [00:00<00:00, 1974363.63it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████████████████████████████████

	Aleppo completed in: 0.381 minutes





Starting Auckland
-------------------------





100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60364/60364 [00:02<00:00, 22273.85it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60364/60364 [00:00<00:00, 233504.51it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57031/57031 [00:00<00:00, 1340048.47it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59664/59664 [00:00<00:00, 1535938.68it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████████████████████████████████

	Auckland completed in: 0.518 minutes





Starting Bucaramanga
-------------------------





...

	Liège completed in: 0.539 minutes





Starting Salt Lake City
-------------------------





100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50917/50917 [00:03<00:00, 16781.67it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50917/50917 [00:00<00:00, 238843.97it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46988/46988 [00:00<00:00, 1223997.49it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50591/50591 [00:00<00:00, 1499307.80it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████████████████████████████████

	Salt Lake City completed in: 0.321 minutes





Starting Wuhan
-------------------------





100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92667/92667 [00:04<00:00, 22245.42it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92667/92667 [00:00<00:00, 320214.84it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82891/82891 [00:00<00:00, 1510951.99it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 92321/92321 [00:00<00:00, 1774816.38it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████████████████████████████████

	Wuhan completed in: 0.67 minutes





~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
Total runtime: 3.332 minutes
